In [8]:
import tensorflow as tf
from google.colab import drive
drive.mount("/content/drive")

project_path = "/content/drive/My Drive/AI - Deep Learning/CNN - Project 1 - R7/"



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
# Read the Dataset
import zipfile as zf
train_zip = zf.ZipFile(project_path+'plant-seedlings-classification.zip')
train_zip.extractall()
!ls

drive  sample_data  sample_submission.csv  test.zip  train.zip


In [10]:
train_zip = zf.ZipFile('train.zip')
train_zip.extractall()
test_zip = zf.ZipFile('test.zip')
test_zip.extractall()
!ls

drive  sample_data  sample_submission.csv  test  test.zip  train  train.zip


In [18]:
!ls train

 Black-grass	    'Common wheat'	'Scentless Mayweed'
 Charlock	    'Fat Hen'		'Shepherds Purse'
 Cleavers	    'Loose Silky-bent'	'Small-flowered Cranesbill'
'Common Chickweed'   Maize		'Sugar beet'


In [0]:
#list of useful imports that  I will use
import os

import cv2
import numpy as np
from glob import glob

In [0]:
train_folder = "train"

train_images = []
classes=[]
missing=0
for class_folder_name in os.listdir(train_folder):
    class_folder_path = os.path.join(train_folder, class_folder_name)
    class_label = class_folder_name
    
    for image_path in glob(os.path.join(class_folder_path, "*.png")):
        image_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if image_bgr is None:
          missing += 1
          continue
        train_images.append(image_bgr)
        
        classes.append(class_label)

In [28]:
len(train_images)

4750

In [0]:
def resize_images(img):

  img = np.array(img).astype(np.uint8)
  res = cv2.resize(img,(128,128), interpolation = cv2.INTER_CUBIC)
  return res

In [0]:
#save resized images into images.
train_images = [resize_images(img) for img in train_images]

In [31]:
train_images[0].shape

(128, 128, 3)

In [37]:
len(train_images)

4750

In [0]:
test_folder = "test"

test_images = []
missing=0
# for image_path in os.listdir(test_folder):
#     class_folder_path = os.path.join(train_folder, class_folder_name)
    
for image_path in glob(os.path.join(test_folder, "*.png")):
    image_bgr = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image_bgr is None:
      missing += 1
      continue
    test_images.append(image_bgr)

In [0]:
test_images = [resize_images(img) for img in test_images]

In [34]:
test_images[0].shape

(128, 128, 3)

In [36]:
len(test_images)

794

In [0]:
import pandas as pd
dum = pd.get_dummies(classes)

In [0]:
encoded_labels = dum
y_train = dum

In [0]:
import numpy as np
y_train = np.array(y_train)

In [42]:
x_train = train_images
x_train = np.array(x_train)
x_train[0].shape

(128, 128, 3)

In [43]:
from sklearn.model_selection import train_test_split
x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
print (len(x_train2))
print (len(x_val))

3800
950


In [44]:
x_train2[0].shape

(128, 128, 3)

In [0]:
x_train2 = x_train2.reshape(x_train2.shape[0],128,128,3)

In [0]:
x_val = x_val.reshape(x_val.shape[0],128,128,3)

In [47]:
print (x_train2.shape)
print (x_val.shape)

(3800, 128, 128, 3)
(950, 128, 128, 3)


In [48]:
print(y_train2.shape)
print(y_val.shape)

(3800, 12)
(950, 12)


In [49]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3))) 
model.add(MaxPooling2D(pool_size=2))

model.add(Convolution2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Convolution2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128,activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 12, activation = 'softmax')) 

In [0]:
optimizer = adam(lr=0.001)
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 63, 63, 64)        32832     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 31, 31, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 128)       0         
__________

In [59]:
batch_size = 64
nb_epochs = 10
history = model.fit(x_train, y_train, epochs = 25, validation_data = (x_val,y_val),batch_size = batch_size, verbose = 1, initial_epoch=0)

Train on 4750 samples, validate on 950 samples
Epoch 1/25
4750/4750 [==============================] - 8s 2ms/step - loss: 0.0751 - acc: 0.9731 - val_loss: 0.0458 - val_acc: 0.9853
Epoch 2/25
4750/4750 [==============================] - 8s 2ms/step - loss: 0.0629 - acc: 0.9768 - val_loss: 0.0397 - val_acc: 0.9874
Epoch 3/25
4750/4750 [==============================] - 8s 2ms/step - loss: 0.0546 - acc: 0.9815 - val_loss: 0.0375 - val_acc: 0.9905
Epoch 4/25
4750/4750 [==============================] - 8s 2ms/step - loss: 0.0472 - acc: 0.9842 - val_loss: 0.0262 - val_acc: 0.9947
Epoch 5/25
4750/4750 [==============================] - 8s 2ms/step - loss: 0.0899 - acc: 0.9672 - val_loss: 0.0747 - val_acc: 0.9737
Epoch 6/25
4750/4750 [==============================] - 8s 2ms/step - loss: 0.0648 - acc: 0.9785 - val_loss: 0.0312 - val_acc: 0.9884
Epoch 7/25
4750/4750 [==============================] - 8s 2ms/step - loss: 0.0446 - acc: 0.9844 - val_loss: 0.0391 - val_acc: 0.9874
Epoch 8/25
4750

In [61]:
score = model.evaluate(x_val, y_val, verbose=0, batch_size = 38)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.00032510888340766544
Test accuracy: 1.0
